# Violation Limit Crosswalk Exploration

<i> Author: Ryan Treves

Goal: figure out a way to identify the specific limit series from a violation record.

In [65]:
import pandas as pd

# Suppress warning messages
import warnings

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 200
pd.options.display.max_rows = 2000

In [66]:
# Load in data from `viol_limit_preprocessing.ipynb`. Using `viols_prepped_1.csv` as an example.
violations_complete = pd.read_csv('/Users/rtreves/Documents/DMR work/DMR Python projects/ca_mmp/policy_eval/notebooks/viol_limit_matching/viols_prepped_1.csv')

limits = pd.read_csv('/Users/rtreves/Documents/DMR work/DMR Python projects/ca_mmp/policy_eval/notebooks/viol_limit_matching/CA_limits_prepped.csv', dtype={
    'parameter_code': object,
    'perm_feature_nmbr': object,
    'monitoring_location_code': object})

Matching algorithm:

In [67]:
limit_series_unique = ['npdes_permit_id', 'parameter_desc', 'perm_feature_nmbr', 'limit_value_type_code', 'statistical_base_code']

In [ ]:
match_dict = {}
for i in range(0, len(violations_complete)):
    violation = violations_complete.iloc[i]

    # Permit ID match (match required)
    match_dict[violation['VIOLATION ID (VID)']] = limits[limits['npdes_permit_id'] == violation['NPDES# CA#']]

    # Parameter match test: exact match or parameter mentioned in violation description (optional)

    # Before we run the test, spot-fixing a few common parameters will resolve a number of matching issues.
    violation['Parameter'] = violation['Parameter'].replace(',', '')
    violation['Parameter'] = violation['Parameter'].replace(
        'carbonaceous biochemical oxygen demand [cbod] [5-day @ 20 deg. c]', 'bod carbonaceous [5 day 20 c]')
    violation['Parameter'] = violation['Parameter'].replace('biochemical oxygen demand [bod] [5-day @ 20 deg. c]',
                                                            'bod 5-day 20 deg. c')
    violation['Parameter'] = violation['Parameter'].replace('total suspended solids [tss]', 'solids total suspended')
    violation['Parameter'] = violation['Parameter'].replace('total dissolved solids [tds]', 'solids total dissolved')
    if violation['Parameter'] == 'chloride':
        violation['Parameter'] = 'chloride [as cl]'

    # Run Parameter match test (match optional)
    new = match_dict[violation['VIOLATION ID (VID)']][
        (match_dict[violation['VIOLATION ID (VID)']]['parameter_desc'] == violation['Parameter']) |
        (match_dict[violation['VIOLATION ID (VID)']]['parameter_desc'].apply(
            lambda x: x in violation['VIOLATION DESCRIPTION'])) |
        (match_dict[violation['VIOLATION ID (VID)']]['parameter_desc'].apply(lambda x: violation['Parameter'] in x))]
    if len(new) > 0:
        match_dict[violation['VIOLATION ID (VID)']] = new

    # Outfall match (match optional)
    if not pd.isna(violation['Monitoring Location']):
        new = match_dict[violation['VIOLATION ID (VID)']][
            match_dict[violation['VIOLATION ID (VID)']]['perm_feature_nmbr'].apply(
                lambda x: x in violation['Monitoring Location'])]
    if len(new) > 0:
        match_dict[violation['VIOLATION ID (VID)']] = new

    # Limit test (match optional)
    new = match_dict[violation['VIOLATION ID (VID)']][
        match_dict[violation['VIOLATION ID (VID)']]['limit_value_nmbr'] == violation['Limit']]
    if len(new) > 0:
        match_dict[violation['VIOLATION ID (VID)']] = new

    # We only want to match to a limit series, not to a specific limit
    match_dict[violation['VIOLATION ID (VID)']].drop_duplicates(subset='limit_series_id', inplace=True)

What is the distribution of number of matches per violation?

What is the distribution of number of matches per violation?

In [69]:
key_lengths = []
for key in match_dict.keys():
    key_lengths.append(len(match_dict[key]))
pd.DataFrame(pd.Series(key_lengths).value_counts(normalize=True)).head()

,0
2,0.307236
1,0.292731
3,0.169606
10,0.044998
5,0.035108


What is the distribution of number of unique parameter values represented in the matches for each violation?

In [70]:
key_unique_parameters = []
for key in match_dict.keys():
    key_unique_parameters.append(len(match_dict[key].drop_duplicates(subset='parameter_desc')))
pd.DataFrame(pd.Series(key_unique_parameters).value_counts(normalize=True)).head()

,0
1,0.872919
2,0.031811
43,0.019614
5,0.008571
218,0.005439


,0
1,0.872919
2,0.031811
43,0.019614
5,0.008571
218,0.005439


Manually review violations matched to one series:

In [71]:
exact_match_keys = pd.DataFrame()
for key in match_dict.keys():
    if len(match_dict[key]) == 1:
        exact_match_keys = pd.concat([exact_match_keys, violations_complete[violations_complete['VIOLATION ID (VID)'] == key][
            ['VIOLATION ID (VID)', 'NPDES# CA#', 'Parameter', 'Limit', 'Result', 'Monitoring Location']]], axis=0)
exact_match_keys = exact_match_keys.reset_index(drop=True)

exact_matches = pd.DataFrame()
for VID in exact_match_keys['VIOLATION ID (VID)']:
    exact_matches = pd.concat([exact_matches, match_dict[VID][limit_series_unique + ['limit_value_nmbr']]], axis=0)
exact_matches = exact_matches.reset_index(drop=True)

In [72]:
pd.concat([exact_match_keys[['VIOLATION ID (VID)', 'Parameter']],
           exact_matches[['parameter_desc']]], axis=1).head()

,VIOLATION ID (VID),Parameter,parameter_desc
0,897482,total coliform,coliform total general
1,897483,total coliform,coliform total general
2,897484,total coliform,coliform total general
3,897500,total coliform,coliform total general
4,897501,total coliform,coliform total general


Note: some reasons for incorrect single matches:
- DMR parameter is a substring of violation Parameter (e.g. - 'dichloromethane' in 'dibromodichloromethane')
- Violation parameter wasn't in the set of all parameters in the DMR data for that permittee (e.g., 'chronic toxicity')

Manually reviewing violations matched to multiple series:

In [73]:
many_match_keys = pd.DataFrame()
for key in match_dict.keys():
    if len(match_dict[key]) > 1:
        many_match_keys = pd.concat([many_match_keys, violations_complete[violations_complete['VIOLATION ID (VID)'] == key][
            ['VIOLATION ID (VID)', 'NPDES# CA#', 'Parameter', 'Limit', 'Result', 'Monitoring Location']]], axis=0)
many_matches = pd.DataFrame()
for VID in many_match_keys['VIOLATION ID (VID)']:
    to_display = match_dict[VID][limit_series_unique]
    to_display['VID'] = VID
    many_matches = pd.concat([many_matches, to_display], axis=0)

In [74]:
many_match_keys.head()

,VIOLATION ID (VID),NPDES# CA#,Parameter,Limit,Result,Monitoring Location
1,897485,CA8000383,total coliform,23.00,70.0,M-001
2,897502,CA8000383,total coliform,23.00,80.0,M-001
11,856456,CA0077712,chlorine total residual,0.02,2.0,NaN
17,863738,CA0077712,acute toxicity,70.00,55.0,NaN
19,869645,CA0077712,aluminum total,71.00,130.0,NaN


In [75]:
many_matches.head()

,npdes_permit_id,parameter_desc,perm_feature_nmbr,limit_value_type_code,statistical_base_code,VID
98349,CA8000383,coliform total general,001,C2,7C,897485
177142,CA8000383,coliform total general,001,C1,3E,897485
98349,CA8000383,coliform total general,001,C2,7C,897502
177142,CA8000383,coliform total general,001,C1,3E,897502
40862,CA0077712,chlorine total residual,001,C3,1I,856456


Note: some reasons for multiple matches:
- Multiple base codes
- No DMR record of the violation parameter (manually checked)
- Others?

### Stepping back: how much time do we think that manual matching will take, using this algorithm?

Let's say we're able to link all EPLs to their 11,427 associated violations, and they all contain data on NPDES permittee ID. At this moment we haven't calculated the number of MMP-eligible effluent violations associated with NPDES permittees that did NOT result in EPLs (a candidate control group); however, we know there are 1,017 facilities in said group (see MMP universe file in EPA > CA MMP google drive). Assuming the facility:violation ratio is the same for both groups (which it certainly may not be), we could estimate that there would be about 17,000 violations in this control group. Thus, we would have a total of 28,427 violations to match.

Assuming we don't care about why data in the 'Parameter', 'Limit', 'Monitoring Location', and 'Result' is missing, about half (14,213) of these violations will have complete data for these columns. Running this many violations through the algorithm above won't take more than an hour, maximum - likely less.
If we wanted to spot-check every single violation and flag incorrect matches, I would estimate that would take one person 5 minutes per 50 violations = approximately 30 hours of work (including short breaks) to check all 14,213 violations.
If we only wanted to spot-check and flag violations for which there was a parameter mismatch, I would guess that the whole process could be shortened by about 4x to ~8 hours of work.
If we assume a similar rate of incorrect matches as in the exploration above (~4-6%), I would estimate the additional work of correcting ~750 incorrect matches to take one person 1 minute per violation = approximately 15 hours of work (including short breaks) to either correct the incorrect matches or throw the violations out as unmatchable.

In conclusion, my guess for the time demand of matching all of our treatment and control violations to DMR limit series (ignoring outfall number and base code) is <b>23-45 person-hours of work (depending on how confident we want to be in our matches).

Remaining questions:
- Why are so many violations missing data for 'Parameter', 'Limit', 'Monitoring Location', and 'Result'?
- Why do a small number of violations have NPDES permit ID values not appearing in the CA DMR data?
- Do we want to keep iterating on the above algorithm to try and increase the rate of correct matches?
- What do we want to do about violation Parameters such as 'chronic toxicity' that don't appear in the CA DMR data?
- How easy is it really for us to match EPLs to violations?